# Homework 3

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_full = pd.read_parquet("D:\zoomcamp\myproject\stock-market-analytics\module3\downloadsfchkomj5.part")
df_full.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.088542,0.101563,0.088542,0.097222,0.060163,1.031789e+09,MSFT,1986,1986-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.097222,0.102431,0.097222,0.100694,0.062311,3.081600e+08,MSFT,1986,1986-03-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.100694,0.103299,0.100694,0.102431,0.063386,1.331712e+08,MSFT,1986,1986-03-01,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.102431,0.103299,0.098958,0.099826,0.061774,6.776640e+07,MSFT,1986,1986-03-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.099826,0.100694,0.097222,0.098090,0.060700,4.789440e+07,MSFT,1986,1986-03-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV

In [4]:
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

C:\Users\Garjita\AppData\Local\Temp\ipykernel_26064\912909511.py:1: RuntimeWarning: divide by zero encountered in log
  df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))


In [5]:
# manually defined features
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

In [6]:
# All Supported Ta-lib indicators: https://github.com/TA-Lib/ta-lib-python/blob/master/docs/funcs.md

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

In [7]:
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')


Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [8]:
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS', 'DGS1', 'DGS5', 'DGS10']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

# CHECK: NO OTHER INDICATORS LEFT
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]
OTHER

['growth_future_5d', 'is_positive_growth_5d_future']

In [9]:
df_full.Ticker.nunique()

33

In [10]:
# truncated df_full with 25 years of data (and defined growth variables)
df = df_full[df_full.Date>='2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 203 entries, Open to ln_volume
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 239.7+ MB


In [11]:
# dummy variables are not generated from Date and numeric variables
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

In [12]:
# Generate dummy variables (no need for bool, let's have int32 instead)
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')


In [13]:
# get dummies names in a list
DUMMIES = dummy_variables.keys().to_list()

## Question 1 (1 point): Dummies on Month and Week-of-Month
Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy with the binary outcome variable is_positive_growth_5d_future?

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'. Once you've generated the new set of variables, find the most correlated (in absolute value) one with is_positive_growth_5d_future and round it to 3 digits after the comma.

Suggested path to a solution:

[Source] Use this formula to get the week of month for the datetime variable d: (d.day-1)//7+1 Define a new string variable for all month-week_of_month combinations. Append it to the CATEGORICAL features set. You should have 5 variables treated as CATEGORICAL now: 'Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom'. In the end, you should get 115 dummy features, including 60 (=12*5) week_month_of_week dummies. Use pandas.get_dummies() to generate dummies. Use pandas.DataFrame.corr() function (also used in [Code Snippet 1]) to get correlations with is_positive_growth_5d_future, filter out only variables representing the new dummy set, and sort it by absolute values (you can define a new column "abs_corr" in the dataframe with correlations), and find the highest value (among the new dummies features set). NOTE: new dummies will be used as features in the next tasks, please leave them in the dataset.

**Question 1. [Dummies on Month and Week-of-Month] Top correlation of a new dummy with a boolean prediction variable? (1 point)**

- 0.025

- 0.035

- 0.045

- 0.055

In [14]:
# TODO 1: define more categorical features, e.g. all combinations for <September+weekday>  (you'll see that September is actually an important dummy in one of the models)
week_of_month  = ((df["Date"].dt.day - 1) // 7) + 1
month_week_of_month = df["Date"].dt.month_name().str.cat(week_of_month.astype(str), sep="_w")
month_week_of_month

3490    January_w1
3491    January_w1
3492    January_w1
3493    January_w1
3494    January_w1
           ...    
5422      April_w5
5423        May_w1
5424        May_w1
5425        May_w1
5426        May_w1
Name: Date, Length: 182675, dtype: object

In [15]:
month_week_of_month_dummy = pd.get_dummies(month_week_of_month)
correlations = month_week_of_month_dummy.corrwith(df["is_positive_growth_5d_future"])

In [16]:
top_correlation = round(abs(correlations[[correlations.abs().idxmax()]]), 3)

In [17]:
print("Top correlation of a new dummy with a boolean prediction variable:\n", top_correlation)

Top correlation of a new dummy with a boolean prediction variable:
 September_w3    0.035
dtype: float64


### Answer of Question 1 : 0.035

## Question 2 (2 points): Define new "hand" rules on macro and technical indicators variables
What is the precision score for the best of the NEW predictions (pred3 or pred4), rounded to 3 digits after the comma?

Let's utilize the knowledge from the visualised tree (clf10) (Code Snippet 5: 1.4.4 Visualisation):

You're asked to define two new 'hand' rules (leading to 'positive' subtrees):

pred3_manual_gdp_fastd: (gdppot_us_yoy <= 0.027) & (fastd >= 0.251) pred4_manual_gdp_wti_oil: (gdppot_us_yoy >= 0.027) & (growth_wti_oil_30d <= 1.005) Extend the Code Snippet 3 (Manual "hand rule" predictions): Calculate and add new rules (pred3 and pred4) to the dataframe.You should notice that one of the predictions doesn't have any positive predictions on TEST dataset (while it has many on TRAIN+VALIDATION).

Debug: check in the new_df and the original dataset/data generation process that we didn't make any mistakes during the data transformation step.

Explain why this can happen even if there are no errors in the data features.

As a result, write down the precision score for the remaining predictor (round to three decimal points). E.g. if you have 0.57897, your answer should be 0.579.

**Question 2. [Define new 'hand' rules] What's the precision score for the best new rule? (2 points)**

- 0.575

- 0.591

- 0.612

- 0.555

In [18]:
# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [19]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 239 entries, growth_1d to ticker_type_US
dtypes: float64(121), int32(117), int64(1)
memory usage: 253.0 MB


In [20]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

In [21]:
min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)

In [22]:
# remove the "segmentation" problem (warning message on df performance after many joins and data transformations)
new_df = df_with_dummies.copy()

In [23]:
# Split the data into training and testing sets based on the split date
features_list = NUMERICAL+DUMMIES
to_predict = 'is_positive_growth_5d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}')

#
# Can't have +-inf values . E.g. ln(volume)=-inf when volume==0 => substitute with 0

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}')

#
# you may want to remove 1-2% outliers based on percentile ==> not used here in Decision Trees
def remove_outliers_percentile(X, lower_percentile=1, upper_percentile=99):
    """
    Remove outliers from the input array based on percentiles.

    Parameters:
    - X: Input array (NumPy array or array-like)
    - lower_percentile: Lower percentile threshold (float, default=1)
    - upper_percentile: Upper percentile threshold (float, default=99)

    Returns:
    - Array with outliers removed
    """
    lower_bound = np.percentile(X, lower_percentile, axis=0)
    upper_bound = np.percentile(X, upper_percentile, axis=0)
    mask = np.logical_and(np.all(X >= lower_bound, axis=1), np.all(X <= upper_bound, axis=1))
    return X[mask]

# Example usage:
# Assuming X is your input data
# filtered_X = remove_outliers_percentile(X, lower_percentile=1, upper_percentile=99)

#
X_train_imputed = X_train # we won't use outliers removal to save more data to train: remove_outliers_percentile(X_train)
X_test_imputed = X_test # we won't use outliers removal to save more data to test: remove_outliers_percentile(X_test)

#
y_train = X_train_imputed[to_predict]
y_test = X_test_imputed[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train_imputed[to_predict]
del X_test_imputed[to_predict]

#
# Define new 'hand' rules
pred3_manual_gdp_fastd = (test_df['gdppot_us_yoy'] <= 0.027) & (test_df['fastd'] >= 0.251)
pred4_manual_gdp_wti_oil = (test_df['gdppot_us_yoy'] >= 0.027) & (test_df['growth_wti_oil_30d'] <= 1.005)

# Add new rules to the dataframe
test_df['pred3'] = pred3_manual_gdp_fastd.astype(int)
test_df['pred4'] = pred4_manual_gdp_wti_oil.astype(int)

# Evaluate the performance of pred3 and pred4 on the test dataset
from sklearn.metrics import precision_score

# Ensure alignment between test dataset and predictions
y_test = y_test[:len(test_df)]

# Evaluate the performance of pred3 and pred4 on the test dataset
precision_pred3 = precision_score(y_test, test_df['pred3'])
precision_pred4 = precision_score(y_test, test_df['pred4'])

# Choose the predictor with the highest precision score
best_precision = max(precision_pred3, precision_pred4)

# Round the precision score to 3 decimal points
rounded_precision = round(best_precision, 3)

print("Precision score for the best predictor:", rounded_precision)

length: X_train (152846, 242),  X_test (29829, 242)
length: X_train_imputed (152846, 242),  X_test_imputed (29829, 242)
Precision score for the best predictor: 0.555


D:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Answer of Question 2 : 0.555

## Question 3 (1 point): Unique correct predictions from a 10-levels deep Decision Tree Classifier (pred5_clf_10)
What is the total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4)?

NOTE: please include random_state=42 to Decision Tree Classifier init function (line clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)) to ensure everyone gets the same results.

Suggested solution:

Step1: Rewrite the '1.4.3 Inference for a decision tree' piece for the Decision Tree Classifier with max_depth=10 (clf_10), so that you fit the model on TRAIN+VALIDATION sets (unchanged from the lecture), but predict on the whole set X_all (to be able to define a new column 'pred5_clf_10' in the dataframe new_df). Here is the link with explanation. It will solve the problem in 1.4.5 when predictions were made only for Test dataset and couldn't be easily joined with the full dataset.

Step2: Once you have it, define a new column 'only_pred5_is_correct' similar to 'hand' prediction rules with several conditions: is_positive_growth_5d_future AND is_correct_pred5 should be equal 1, while all other predictions is_correct_pred0..is_correct_pred4 should be equal to 0.

Step3: Convert 'only_pred5_is_correct' column from bool to int, and find how many times it is equal to 1 in the TEST set. Write down this as an answer.

ADVANCED: define a function that can be applied to the whole row of predictions (a few examples of pandas-apply-row-functions) and can find whether some prediction 'predX' (where X is one of the predictions) is uniquely correct. It should work even if there are 100 predictions available, so that you don't define manually the condition for 'predX'.

**Question 3. [Unique correct predictions for pred5_clf_10] Wha'ts the total number of records of 'unique correctness' on TEST? (1 point)**

- 1

- 5

- 10

- 15

- 20

In [24]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

### 1.4.1) Define dataframes AND perform data cleaning
* define X_train (dataframe), X_test (dataframe), y_train (series), y_test (series)
* replace +-inf. with 0
* fill NaNs with 0 (you can drop it too, but will loose a lot of data in our case
* remove 1-2% outliers (in each dimension, or only in variable to_predict :: we won't use it for a Decision Tree

In [25]:
# Decision Tree doesn't like too large and inf. values
import numpy as np

def remove_infinite_values(X):
    """
    Remove infinite values from the input array.

    Parameters:
    - X: Input array (NumPy array or array-like)

    Returns:
    - Array with infinite values removed
    """
    return X[np.isfinite(X).all(axis=1)]

# Example usage:
# Assuming X is your input data
# filtered_X = remove_infinite_values(X)

In [26]:
# Split the data into training and testing sets based on the split date
features_list = NUMERICAL+DUMMIES
to_predict = 'is_positive_growth_5d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}')


length: X_train (152846, 242),  X_test (29829, 242)


In [27]:
# Can't have +-inf values . E.g. ln(volume)=-inf when volume==0 => substitute with 0

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}')

length: X_train_imputed (152846, 242),  X_test_imputed (29829, 242)


In [28]:
# you may want to remove 1-2% outliers based on percentile ==> not used here in Decision Trees
def remove_outliers_percentile(X, lower_percentile=1, upper_percentile=99):
    """
    Remove outliers from the input array based on percentiles.

    Parameters:
    - X: Input array (NumPy array or array-like)
    - lower_percentile: Lower percentile threshold (float, default=1)
    - upper_percentile: Upper percentile threshold (float, default=99)

    Returns:
    - Array with outliers removed
    """
    lower_bound = np.percentile(X, lower_percentile, axis=0)
    upper_bound = np.percentile(X, upper_percentile, axis=0)
    mask = np.logical_and(np.all(X >= lower_bound, axis=1), np.all(X <= upper_bound, axis=1))
    return X[mask]

# Example usage:
# Assuming X is your input data
# filtered_X = remove_outliers_percentile(X, lower_percentile=1, upper_percentile=99)

In [29]:
X_train_imputed = X_train # we won't use outliers removal to save more data to train: remove_outliers_percentile(X_train)
X_test_imputed = X_test # we won't use outliers removal to save more data to test: remove_outliers_percentile(X_test)

In [30]:
# same shape
print(f'length: X_train_imputed {X_train_imputed.shape},  X_test_imputed {X_test_imputed.shape}')

length: X_train_imputed (152846, 242),  X_test_imputed (29829, 242)


In [31]:
y_train = X_train_imputed[to_predict]
y_test = X_test_imputed[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train_imputed[to_predict]
del X_test_imputed[to_predict]

### 1.4.2 Estimation of a Decision Tree model

In [32]:
# INPUTS:
# X_train_imputed : CLEAN dataFrame with only numerical features (train+validation periods)
# X_test_imputed : CLEAN dataFrame with only numerical features (test periods)

# y_train : true values for the train period
# y_test  : true values for the test period

In [33]:
# estimation/fit function (using dataframe of features X and what to predict y) --> optimising total accuracy
# max_depth is hyperParameter
def fit_decision_tree(X, y, max_depth=20):
# Initialize the Decision Tree Classifier
  clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

  # Fit the classifier to the training data
  clf.fit(X, y)
  return clf, X.columns

In [34]:
%%time
clf_10, train_columns = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=10)

CPU times: total: 10.8 s
Wall time: 11.8 s


In [35]:
# TODO 3: TRAIN only on train dataset, experiment with trees with depth 1..20 --> find the best one on VALID dataset
#       for the "best" tree model: find precision on the TEST set

### 1.4.3 Inference for a Decision Tree

In [36]:
def predict_decision_tree(clf:DecisionTreeClassifier, df_X:pd.DataFrame, y_true: pd.Series):
  # Predict the target variable on the test data
  y_pred = clf.predict(df_X)

  max_depth = clf.tree_.max_depth
  # Print the maximum depth
  print("Maximum depth of the decision tree:", max_depth)

  # Calculate the accuracy/precision of the model
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  print(f'Accuracy ={accuracy}, precision = {precision}')

  # resulting df
  result_df = pd.concat([df_X, y_true, pd.Series(y_pred, index=df_X.index, name='pred_')], axis=1)

  return result_df

In [37]:
# Feautures importance function to predict future returns (based on the classifier)
# get feature importance from 'clf' (classifier) and 'train_columns' (column names)

def get_importances(clf, train_columns):
  # Assuming clf is your trained DecisionTreeClassifier
  feature_importance = clf.feature_importances_

  # Assuming X_train is your training features
  feature_names = train_columns

  # Create a DataFrame to store feature importance
  feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

  # Sort the DataFrame by importance in descending order
  feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

  # Print or display the feature importance DataFrame
  # print(feature_importance_df)
  return feature_importance_df

In [38]:
# TODO 4: JOIN predictions with the original dataframe (define a new column):
#  so, that there are columns pred_tree_clf10 AND pred_tree_clf20

In [39]:
test_pred = clf_10.predict(X_test_imputed.drop(['Date','Ticker'],axis=1))
tp_series = pd.Series(test_pred, dtype=int)
tp_series.head()

0    1
1    1
2    1
3    1
4    1
dtype: int32

In [40]:
test_mask = new_df["split"] == "test"
new_df.loc[test_mask, "pred5_clf_10"] = tp_series

In [41]:
pred0_to_4 = new_df.loc[test_mask, new_df.columns.str.contains(r"^pred[0-4]")]

In [42]:
manual_all_wrong = (pred0_to_4.values != new_df.loc[test_mask, to_predict].values[:, np.newaxis]).all(axis=1)
pred_5_correct = new_df.loc[test_mask, to_predict].values == new_df.loc[test_mask, "pred5_clf_10"].values

In [43]:
(manual_all_wrong & pred_5_correct).sum()

16168

In [44]:
pred_cols = new_df.columns[new_df.columns.str.contains(r"^pred")].to_list()
new_df.loc[test_mask, pred_cols + [to_predict]][manual_all_wrong & pred_5_correct]


,pred5_clf_10,is_positive_growth_5d_future
8700,1.0,1
8701,1.0,1
8702,1.0,1
8704,1.0,1
8705,1.0,1
...,...,...
5412,1.0,1
5413,1.0,1
5414,1.0,1
5415,1.0,1


In [45]:
predictions_forpred5_clf_10 = new_df.loc[test_mask, pred_cols + [to_predict]][manual_all_wrong & pred_5_correct]

In [46]:
predictions_forpred5_clf_10['is_positive_growth_5d_future']

8700    1
8701    1
8702    1
8704    1
8705    1
       ..
5412    1
5413    1
5414    1
5415    1
5416    1
Name: is_positive_growth_5d_future, Length: 16168, dtype: int64

In [47]:
predictions_forpred5_clf_10.is_positive_growth_5d_future.unique()

array([1, 0], dtype=int64)

### Answer of Question 3 : 1

## Question 4: (2 points) Hyperparameter tuning for a Decision Tree
What is the optimal tree depth (from 1 to 20) for a DecisionTreeClassifier?

NOTE: please include random_state=42 to Decision Tree Classifier init function (line clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)) to ensure consistency in results.

Follow these steps to find the optimal max_depth:

Iterate through max_depth values from 1 to 20. Train the Decision Tree Classifier with the current max_depth parameter. Optionally, visualize how the 'head' of each fitted tree changes with more advanced (=deep) trees. You can use the sklearn.tree.plot_tree() function, or the compact way with the export_text() functionality (Stack Overflow example): from sklearn.tree import export_text tree_rules = export_text(model, feature_names=list(X_train), max_depth=3) print(tree_rules) Calculate the precision score (you can use the function sklearn.metrics.precision_score()) on the TEST dataset for each of the fitted trees. You can also compare it with the precision score on a VALIDATION dataset, which is included to the training phase (to have more data to train on). You should see that the precision score on a VALIDATION set starts to grow with the complexity of a tree (overfit), which isn't seen on the precision score on TEST. Identify the optimal max_depth, where the precision score is the highest on the TEST dataset. Record this value as best_max_depth and submit as an answer. Make predictions on all records (TRAIN+VALIDATION+TEST) and add the new prediction pred6_clf_best to the dataframe new_df. Additionally, compare the precision score of the tuned decision tree with previous predictions. You should observe an improvement (>0.58, or more than 58% precision), indicating that the tuned tree outperforms previous manual "hand" rules and Decision Tree predictions.

ADVANCED: Read more about different aspects of scikit-learn Decision Trees. Draw a line of precision/accuracy vs. max_depth and note whether there's a saturation point of precision/accuracy as max_depth increases. In theory, there should be a trade-off between better fitting (=more complex trees) and generalization.

**Question 4. [Hyperparams tuning] What's the best max_depth for a Decision Tree Classifier? (2 points)**

- 13

- 16

- 15

- 18

- 20

In [48]:
# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = new_df.loc[new_df["split"].isin(['train','validation']), features_list]
y_train = new_df.loc[new_df["split"].isin(['train','validation']), to_predict]
X_test = new_df.loc[new_df["split"] == "test", features_list]
y_test = new_df.loc[new_df["split"] == "test", to_predict]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((152846, 239), (152846,), (29829, 239), (29829,))

In [49]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [50]:
accuracies = np.zeros((20, ))
precisions = np.zeros((20, ))
for i, max_depth in enumerate(range(1, 21)):
    print(f"{max_depth=}")
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracies[i] = accuracy_score(y_test, y_pred)
    precisions[i] = precision_score(y_test, y_pred)
    print(f"{accuracies[i]:.3f}\t{precisions[i]:.3f}")

accuracies, precisions

max_depth=1
0.555	0.555
max_depth=2
0.555	0.555
max_depth=3
0.555	0.555
max_depth=4
0.555	0.555
max_depth=5
0.556	0.555
max_depth=6
0.568	0.571
max_depth=7
0.565	0.568
max_depth=8
0.565	0.568
max_depth=9
0.566	0.570
max_depth=10
0.557	0.565
max_depth=11
0.552	0.573
max_depth=12
0.546	0.567
max_depth=13
0.547	0.570
max_depth=14
0.537	0.578
max_depth=15
0.560	0.588
max_depth=16
0.530	0.566
max_depth=17
0.523	0.566
max_depth=18
0.526	0.570
max_depth=19
0.524	0.567
max_depth=20
0.518	0.562


(array([0.55519796, 0.55536558, 0.55519796, 0.55526501, 0.55560025,
        0.56776962, 0.56502062, 0.56502062, 0.56636159, 0.55724295,
        0.55218076, 0.5455094 , 0.54735325, 0.53746354, 0.56022663,
        0.53025579, 0.52274632, 0.52646753, 0.52418787, 0.51801938]),
 array([0.55519796, 0.55529475, 0.55519796, 0.55523519, 0.55546233,
        0.57117355, 0.56839335, 0.56839335, 0.56989745, 0.56526308,
        0.57271943, 0.56734822, 0.57002243, 0.57824708, 0.58798876,
        0.56574671, 0.56621291, 0.56956025, 0.56704417, 0.56231454]))

In [51]:
range(1, 21)[precisions.argmax()]

15

In [52]:
clf = DecisionTreeClassifier(max_depth=12, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=12, random_state=42)

In [53]:
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

pred6_best_clf = np.concatenate([train_pred, test_pred])
new_df["pred6_best_clf"] = pred6_best_clf
new_df.tail()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,Ticker_SIE.DE,Ticker_TCS.NS,Ticker_TTE,Ticker_V,ticker_type_EU,ticker_type_INDIA,ticker_type_US,split,pred5_clf_10,pred6_best_clf
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1571996.0,LT.NS,2024,April,1,...,0,0,0,0,0,1,0,test,1.0,1
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3748847.0,LT.NS,2024,May,3,...,0,0,0,0,0,1,0,test,1.0,1
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696.0,LT.NS,2024,May,4,...,0,0,0,0,0,1,0,test,1.0,1
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667.0,LT.NS,2024,May,0,...,0,0,0,0,0,1,0,test,1.0,1
5426,3479.399902,3496.000000,3425.000000,3427.750000,3427.750000,3375099.0,LT.NS,2024,May,1,...,0,0,0,0,0,1,0,test,1.0,1


### Answer of Question 4 : 15